## PyMC3 Implementation of Chempy
First create a basic Chempy implementation in Chempy for a single end-time and single star

In [38]:
%pylab inline
import scipy,theano,os,inspect,platform
import pymc3 as pm
import numpy.lib.recfunctions as rcfuncs
import numpy as np
import theano.tensor as tt

## TESTING PARAMETERS
full_checking = False # if True, check assertions in code (will be slower)

# Import useful Chempy functions
os.chdir('/home/oliverphilcox/ChempyMulti/')
from Chempy.solar_abundance import solar_abundances as SolarAbundances
from Chempy.infall import PRIMORDIAL_INFALL as PrimordialInfall

# Define localpath
if platform.system()=='Windows':
    string='\\'
else:
    string='/'

localpath = string.join(os.path.abspath(inspect.getfile(inspect.currentframe())).split(string)[:-1])
localpath += string+'Chempy'+string

# # Define incomplete gamma function as theano operator
# from theano.compile.ops import as_op
# from scipy.special import gammainc

# @as_op(itypes=[tt.dscalar,tt.dscalar],
#        otypes=[tt.dscalar])

# def gammainc_th(shape_par,x):
#     return gammainc(shape_par,x)


Populating the interactive namespace from numpy and matplotlib


In [96]:
### MODELLING PARAMETERS

class Parameters(object):
    """This holds various parameters for the PyMC3 Chempy implementation"""
    
    class general(object):
        # General parameters
        elements_to_trace = ['C', 'Fe', 'H', 'He', 'Mg', 'N', 'Ne', 'O', 'Si']
        gas_reservoir_mass_factor = 1.  # how much more mass does the corona has compared to the integrated SFR
        gas_power = 1. # The Schmidt_exponent (default = 1, i.e. linear)
        sfr_factor_for_cosmic_accretion = 0.000001 # how much more gas should be infalling in the corona compared to the SFR
        

    class imf(object):
        # IMF related parameters
        min_mass = 0.1
        max_mass = 100.
        n_mass_steps = 5000 # number of mass steps used

        # Chabrier parameters
        chabrier_A = 0.852464
        chabrier_B = 0.237912
        chabrier_sigma = 0.69
        chabrier_m_c = 0.079 
        
    class sfr(object):
        # SFR related parameters
        a_parameter = 2 # gamma function shape parameter
        
    class times(object):
        # Chempy start and end time in Gyr
        start = 0
        end = 13.797617 # Planck 2015 age
        time_steps = 28 # number of time steps in simulation
        
        star_age = 4.56 # solar age   
        
        assert (star_age <= 13.0), "Age of the star must be below 13Gyr"
        new_end = end - star_age # time to stop the Chempy run

In [97]:
### PRELOADING

# Load parameters
par = Parameters()

class IMF(object):
    # Preload IMF from Chabrier (2003)
    
    masses = np.linspace(par.imf.min_mass,par.imf.max_mass,par.imf.n_mass_steps)
    masses_low = masses[masses<1.]
    masses_high = masses[masses>1.]
    
    masses = tt._shared(masses) # convert to tensors
    masses_low = tt._shared(masses_low)
    masses_high = tt._shared(masses_high) # convert to tensors
    dn_low = par.imf.chabrier_A/masses_low*tt.exp(-(tt.log10(masses_low/par.imf.chabrier_m_c)**2.)/(2.*par.imf.chabrier_sigma**2.))
    
    
class SFR(object):
    # Preload star formation rate (here a Gamma function with shape parameter a = 2 by default)
    # We assume SFR starts at t=0 here

    # Define evolution time steps
    t = tt._shared(np.linspace(par.times.start, par.times.new_end, par.times.time_steps))
    dt = t[1]-t[0]
    timespan = par.times.new_end - par.times.start
    
    # convenience functions
    tmp_t1 = tt.power(t,par.sfr.a_parameter-1)
    tmp_t2 = tt.exp(-1.*t)
    

class Infall(SFR):
    # Preload infall mass over time class (matched to SFR class)
    # This inherits time vector from SFR class
    # We assume SFR-related infall here
    def __init__(self):
        self.infall = tt.zeros_like(self.t)


In [211]:
# Pre-load modules
imf = IMF()
sfr = SFR()
infall = Infall()

# Load solar abundances from Chempy
solar_abundances = SolarAbundances()
solar_abundances.Asplund09()

primordial_infall = PrimordialInfall(par.general.elements_to_trace,solar_abundances.table)
primordial_infall.primordial()

In [219]:
full_model = pm.Model()

with full_model:
    
    ## PARAMETERS TO INFER
    class InferenceParameters(object):
        # Main parameters
        alpha_imf = pm.Normal('imf-slope',mu=-2.3,sd=0.3) # high-mass IMF slope
        log10_N_1a = pm.Normal('log10-N-1a',mu=-2.89,sd=0.3) # SN-1a normalization
        
        log10_sfe = pm.Normal('log10-sfe',mu=-0.3,sd=0.1) # the SFE for a linear Kennicut-Schmidt law
        
        log10_sfr_scale = pm.Normal('log10-sfr-scale',mu=0.55,sd=0.1) 
        x_out = pm.Normal('x-out',mu=0.5,sd=0.1) # fractional enrichment goes into the corona
        
        
        # Derived parameters
        N_1a = tt.pow(10.,log10_N_1a)
        sfr_scale = tt.pow(10.,log10_sfr_scale)
        sfe = tt.pow(10.,log10_sfe)
        
    inference = InferenceParameters()
    
    ## IMF (Chabrier 2003)
    imf.dn_high = par.imf.chabrier_B*imf.masses_high**inference.alpha_imf
    imf.dn = tt.concatenate([imf.dn_low,imf.dn_high])*imf.masses  # number of stars produced per unit mass
    imf.dm = imf.dn/imf.dn.sum() # mass of stars produced per unit mass
    imf.dn = imf.dm/imf.masses
    
    ## SFR (Gamma function)
    
    # First compute normalized gamma function SFR up to the correct end time
    sfr.sfr = sfr.tmp_t1*tt.power(sfr.tmp_t2,1./inference.sfr_scale) # unnormalized gamma function
    sfr.sfr = sfr.sfr/(sfr.sfr.sum())
    
    print('is this SFR normalization correct?')
    
    ## NB: We leave first SFR step as ZERO here
    #sfr.sfr = tt.set_subtensor(sfr.sfr[0],1e-10) # stop the first value being zero to avoid later errors
    
    
#     # Now compute the total mass formed in this simulation (normalized to unit mass formed in universe age)
#     total_mass = gammainc_th(theano.shared(1.*par.sfr.a_parameter),par.times.new_end/inference.sfr_scale)
#     total_mass_univ = gammainc_th(theano.shared(1.*par.sfr.a_parameter),par.times.end/inference.sfr_scale) # total mass formed in universe age
#     total_mass /= total_mass_univ

#     # Check we form enough stars
#     mean_sfr = total_mass_univ/par.times.end
#     sfr_at_end = sfr.sfr[-1]
#     tt.opt.Assert(sfr_at_end>0.05*mean_sfr)

is this SFR normalization correct?


In [252]:
class AbundanceMatrix(object):
    # Preload important quantities for holding the chemical evolution properties of Chempy
    # self.ism contains the ISM evolution, self.gas_reservoir contains the corona evolution 
    # and self.sn2/sn1a/agb_feedback has individual process enrichments
    
    class ISM(object):
        """Class to hold useful quantities for the ISM at each timestep"""
        def __init__(self,elements,time_steps):
            self.mass_fractions = tt.zeros((time_steps,len(elements)))
            self.gas = tt.zeros(time_steps)
            self.Z = tt.zeros(time_steps)
            #self.feedback = tt.zeros(time_steps)
            #self.mass_in_remnants = tt.zeros(time_steps)
    
    class Corona(object):
        """Class to hold useful quantities for the gas reservoir (corona) at each timestep"""
        def __init__(self,elements,time_steps):
            self.mass_fractions = tt.zeros((time_steps,len(elements)))
            self.gas = tt.zeros(time_steps)
            self.Z = tt.zeros(time_steps)
            
            
    def __init__(self,SFR,Infall,PrimordialInfall):
        # Copy useful variables
        ##TODO: do we need all these?
        self.t = SFR.t
        self.dt = SFR.dt
        self.infall = Infall.infall
        self.elements = par.general.elements_to_trace
        self.infall_symbols = PrimordialInfall.symbols
        self.infall_fractions = PrimordialInfall.fractions
        self.gas_power = par.general.gas_power
        self.sfr_factor_for_cosmic_accretion = par.general.sfr_factor_for_cosmic_accretion
        
        ##TODO: DO WE NEED THESE?
        #self.cosmic_accretion_elements = PrimordialInfall.symbols
        #self.cosmic_accretion_element_fractions = PrimordialInfall.fractions
        
        # Define ISM class
        self.ism = self.ISM(self.elements,par.times.time_steps)
        self.corona = self.Corona(self.elements,par.times.time_steps)
        
        # Get indices of elements which are metals:
        metal_list=[]
        for e,el in enumerate(self.elements):
            if el not in ['H','He']:
                metal_list.append(e)
        self.metal_list = tt._shared(np.array(metal_list))
        
        # Add initial gas composition
        # (since we have no initial gas and SFR=0 at t=0 this is independent of free parameters)
        starting_gas = par.general.gas_reservoir_mass_factor
        self.corona.gas=tt.set_subtensor(self.corona.gas[0],starting_gas)
        self.corona.mass_fractions=tt.set_subtensor(self.corona.mass_fractions[0,:],starting_gas*self.infall_fractions)

        # Compute initial metallicity (zero for ISM)
        self.corona.Z = tt.set_subtensor(self.corona.Z[0],self.corona.mass_fractions[0,self.metal_list].sum()/starting_gas)

        
        #self.reservoir = self.CubeData(self.elements)#
        
        #self.ism.feedback[0] = 0.
        


In [253]:
cube=AbundanceMatrix(sfr,infall,primordial_infall)

/home/oliverphilcox/anaconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])


In [248]:
with full_model:
    # Check modeling assumptions
    tt.opt.Assert(sfr.sfr[0]==0.)
    
    ## AbundanceMatrix class
    cube.sfr = sfr.sfr
    cube.starformation_efficiency = inference.sfe*cube.dt
    
    # About to start SSP_wrap(a) in Chempy_all_times()
    pass

/home/oliverphilcox/anaconda3/lib/python3.7/site-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])


## Chempy testing

In [170]:
%pylab inline
import os
os.chdir('/home/oliverphilcox/ChempyMulti/')
from Chempy.parameter import ModelParameters
a=ModelParameters()
from Chempy.sfr import SFR
import Chempy
ch_sfr = SFR(par.times.start,par.times.end,par.times.time_steps)
ch_sfr.gamma_function(S0=1,a_parameter=2,loc=0,scale=10.**0.55)

Populating the interactive namespace from numpy and matplotlib
